<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/gnn/nb2_code_challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Before you start: don't forget to change your runtime

You might need to change the run time to "GPU" in order to run your code successfully (under Runtime -> Change runtime type).

### Code Challenge Question

Using the `pubmed_dataset_student.h5` that has been been randomly split into train, val and test, modify the existing `GAT` archicture to get 80% accuracy. Bonus points for anyone that manages to get over 80% accuracy! 

HINT: Experiment with different types of activation functions, neighbourhood sizes and convolutional layers. 

In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric
!pip install git+https://github.com/Aggregate-Intellect/tutorial-notebook-utils.git

     |████████████████████████████████| 2.6MB 6.0MB/s 
     |████████████████████████████████| 1.5MB 6.3MB/s 
     |████████████████████████████████| 215kB 8.8MB/s 
     |████████████████████████████████| 235kB 14.6MB/s 
     |████████████████████████████████| 2.2MB 15.1MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
  Cloning https://github.com/Aggregate-Intellect/tutorial-notebook-utils.git to /tmp/pip-req-build-stl607da
  Running command git clone -q https://github.com/Aggregate-Intellect/tutorial-notebook-utils.git /tmp/pip-req-build-stl607da
  Created wheel for ai-notebook-utils: filename=ai_notebook_utils-0.0.1-cp37-none-any.whl size=2051 sha256=27f025e523b469e6c43f068b100b01469a37a37eeb78140e4478627a8ec7d982
  Stored in directory: /tmp/pip-ephem-wheel-cache-6t3ge0k3/wheels/41/c0/dd/c53905407cd49b60d4b0d9f657882a17567f4d6af20ea21146
Successfully built ai-notebook-utils


In [2]:
from ai_notebook_utils import *
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import SGConv, GATConv

In [3]:
seed=99
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
!wget https://ai-notebook-public-assets.s3.amazonaws.com/graphnn/codechallenge/nb2/pubmed_dataset_student.h5
data_pubmed = torch.load('pubmed_dataset_student.h5')
num_classes = len(data_pubmed.y.unique())

--2021-05-01 14:23:49--  https://ai-notebook-public-assets.s3.amazonaws.com/graphnn/codechallenge/nb2/pubmed_dataset_student.h5
Resolving ai-notebook-public-assets.s3.amazonaws.com (ai-notebook-public-assets.s3.amazonaws.com)... 52.217.108.68
Connecting to ai-notebook-public-assets.s3.amazonaws.com (ai-notebook-public-assets.s3.amazonaws.com)|52.217.108.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41070311 (39M) [application/x-www-form-urlencoded]
Saving to: ‘pubmed_dataset_student.h5’

pubmed_dataset_stud 100%[===================>]  39.17M  30.6MB/s    in 1.3s    

2021-05-01 14:23:50 (30.6 MB/s) - ‘pubmed_dataset_student.h5’ saved [41070311/41070311]



In [13]:
data_pubmed.num_nodes

19717

In [5]:
def compute_accuracy(model, data, mask):
  # Set the model.training attribute to False
  model.eval()
  logprob = model(data)
  y_pred = logprob[mask].max(1)[1]
  y_true=data.y[mask]
  acc = y_pred.eq(y_true).sum()/ mask.sum().float()
  return acc.item()

In [6]:
def predict(model, data):
    #acc_test = compute_accuracy(model, data, data.test_mask)
    logprob = model(data)
    y_pred = logprob[data.test_mask].max(1)[1]
    if torch.cuda.is_available():
      y_pred = y_pred.cpu().detach().numpy().reshape(-1,)
    else:
      y_pred = y_pred.numpy().reshape(-1,)
    return y_pred

In [7]:
# TODO: complete your code here
class GATNet(torch.nn.Module):
  def __init__(self, data, heads_layer1, 
               heads_layer2, dropout, dropout_alphas):
    pass
  
  def forward(self, data):
    pass

In [8]:
def train(model, data, optimizer):
  # Set the model.training attribute to True
  model.train() 

  # Reset the gradients of all the variables in a model
  optimizer.zero_grad() 

  # Get the output of the network. The output is a log probability of each
  log_softmax = model(data) 

  labels = data.y # Labels of each node

  # Use only the nodes specified by the train_mask to compute the loss.
  nll_loss = F.nll_loss(log_softmax[data.train_mask], labels[data.train_mask])
  
  #Computes the gradients of all model parameters used to compute the nll_loss
  #Note: These can be listed by looking at model.parameters()
  nll_loss.backward()

  # Finally, the optimizer looks at the gradients of the parameters 
  # and updates the parameters with the goal of minimizing the loss.
  optimizer.step() 
  

In [9]:
@torch.no_grad() # Decorator to deactivate autograd functionality  
def test(model, data):
  acc_train = compute_accuracy(model, data, data.train_mask)
  acc_val = compute_accuracy(model, data, data.val_mask)
  acc_test = compute_accuracy(model, data, data.test_mask)
  return acc_train, acc_val, acc_test

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_pubmed= data_pubmed.to(device)

# TODO: pass in necessary parameter for GATNet
model_pubmed_gat = GATNet().to(device)

optimizer = torch.optim.Adam(model_pubmed_gat.parameters(), lr=0.005, weight_decay=1e-3)

TypeError: ignored

In [ ]:
for epoch in range(1, 201):
    train(model_pubmed_gat, data_pubmed, optimizer)
    if epoch %10 ==0:
      log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test {:.4f}'
      train_acc, val_acc, test_acc = test(model_pubmed_gat,data_pubmed)
      #print(train_acc, val_acc, test_acc)
      print(log.format(epoch, train_acc, val_acc, test_acc))

In [ ]:
# save model test results and send it off to AISC for evaluation
answer = predict(model_pubmed_gat, data_pubmed) 

(885,)

In [ ]:
score_answer("graphnn-2020::nb2-codechallenge1::pubmed", answer)

Submission result:  {"accuracy": 0.7909604519774012, "passed": true}
